In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "mfcc_data.json"

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfccs"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y

In [2]:
# load data
X, y = load_data(DATA_PATH)

Data succesfully loaded!


In [3]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# build network topology
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    keras.layers.Dense(512, activation='relu'),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu'),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu'),

    # output layer
    keras.layers.Dense(10, activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 8620)              0         
                                                                 
 dense (Dense)               (None, 512)               4413952   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 4,562,378
Trainable params: 4,562,378
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

Epoch 1/50
22/22 [==============================] - 2s 63ms/step - loss: 23.8675 - accuracy: 0.2132 - val_loss: 9.3465 - val_accuracy: 0.3167
Epoch 2/50
22/22 [==============================] - 1s 58ms/step - loss: 6.3945 - accuracy: 0.4120 - val_loss: 6.8754 - val_accuracy: 0.3367
Epoch 3/50
22/22 [==============================] - 1s 53ms/step - loss: 3.1583 - accuracy: 0.5937 - val_loss: 7.3428 - val_accuracy: 0.3667
Epoch 4/50
22/22 [==============================] - 1s 59ms/step - loss: 1.6202 - accuracy: 0.7067 - val_loss: 5.9118 - val_accuracy: 0.3933
Epoch 5/50
22/22 [==============================] - 1s 57ms/step - loss: 0.7978 - accuracy: 0.8369 - val_loss: 6.3498 - val_accuracy: 0.4467
Epoch 6/50
22/22 [==============================] - 1s 56ms/step - loss: 0.6214 - accuracy: 0.8555 - val_loss: 5.5683 - val_accuracy: 0.4367
Epoch 7/50
22/22 [==============================] - 1s 57ms/step - loss: 0.4325 - accuracy: 0.8941 - val_loss: 5.8909 - val_accuracy: 0.4367
Epoch 8/50
1

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=64)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for tests")
predictions = model.predict(X_test)
for i in range(predictions.shape[0]):
    print(X_test[i],predictions[i])